In [1]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken cohere
!pip install arxiv PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

In [3]:
!pip show langchain

Name: langchain
Version: 0.2.5
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# Simple chain system

1. map out the writing task
2. write a few paragraphs for each chunk


## Utils

In [4]:
# count the tokens utility
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    # encoding = tiktoken.get_encoding(encoding_name)
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# num_tokens_from_string(text_body, "gpt-3.5-turbo")

In [6]:
import textwrap
import os

def wrap_text(text: str, width: int = 120) -> str:
    """Wrap text to a specified width."""
    return '\n'.join(textwrap.wrap(text, width))

def write_string_to_file(filename: str, text: str) -> None:
    """Write a string to a file."""
    with open(filename, 'w') as f:
        f.write(text)

def extract_filename(path_string):
    """Extract filename without extension from a path string."""
    base_name = os.path.basename(path_string)  # Get the filename with extension
    file_name_without_ext = os.path.splitext(base_name)[0]  # Remove the extension
    return file_name_without_ext


## LangChain

In [7]:
! pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.6 MB/s eta 0:00:00


In [8]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain


#### System prompt

In [9]:

context_template="You are a master writer that specializes in creating long 5,000 word articles. Please use all your expertise to approach this task. Output your content in markdown format and include titles and subtitles where relevant."


In [10]:
system_message_prompt = SystemMessagePromptTemplate.from_template(context_template)


#### Human Prompt


In [43]:

human_template="You are a master writer that specializes in creating long 5,000 word articles. Please use all your expertise to approach this task. Output your content in markdown format and include titles and subtitles where relevant. \n\n \
Please generate 15 questions . The questions should what the topic is about and allow you to flesh out each of these questions into a 500 word answer that covers all elements of the question. \n\n Topic: \n\n{topic_description} \
\n\n Once you have generated the 15 questions, think carefully on each of them and write an article that contains a Main title, subtitle for each sub topic and the 500 words (multiptle paragraphs) answer that addresses each of the questions. Don't phrase all sub headings as questions!! \n\n  Questions:\n\n Article: \n\n"


In [44]:

human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=human_template,
            input_variables=["topic_description"],
        )
    )

chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt,
                                                         human_message_prompt])

chat = ChatOpenAI(model_name="gpt-3.5-turbo-16k",
                  temperature=0.2)



In [45]:
chain = LLMChain(llm=chat, prompt=chat_prompt_template)


## Full output

In [46]:
TOPIC_DESCRIPTION = "The pros and cons of regulating AI research and development"

In [48]:
%%time
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = chain.run(TOPIC_DESCRIPTION) # chain.run has been deprecated so chain.invoke is used

    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
    print(f"Calc Total Cost (USD): ${(cb.prompt_tokens/1000)*0.003 + (cb.completion_tokens/1000)*0.004}")

Total Tokens: 1210
Prompt Tokens: 212
Completion Tokens: 998
Total Cost (USD): $0.004628
Calc Total Cost (USD): $0.004628
CPU times: user 77.9 ms, sys: 12.1 ms, total: 90 ms
Wall time: 11.9 s


In [49]:
print(wrap_text(output))

# The Pros and Cons of Regulating AI Research and Development  ## Introduction  Artificial Intelligence (AI) has become
an integral part of our lives, impacting various sectors such as healthcare, finance, and transportation. As AI
continues to advance rapidly, there is an ongoing debate about the need for regulation in AI research and development.
This article explores the pros and cons of regulating AI, addressing key questions and concerns surrounding this topic.
## Pros of Regulating AI Research and Development  ### 1. Ensuring Ethical Development  Regulating AI research and
development can help ensure ethical practices are followed. By establishing guidelines and standards, we can prevent the
creation of AI systems that may cause harm or infringe upon human rights. Ethical considerations such as fairness,
transparency, and accountability can be integrated into the development process, promoting responsible AI deployment.
### 2. Mitigating Bias and Discrimination  AI systems are on

In [50]:
write_string_to_file(f'Article-{TOPIC_DESCRIPTION}.txt', output)

In [51]:
num_tokens_from_string(output, "gpt-3.5-turbo")

998

## Generate questions

#### Prompts


In [52]:
context_template="You are a master writer that specializes in creating long 5,000 word articles. Please use all your expertise to approach this task. Output your content in markdown format and include titles and subtitles where relevant."

system_message_prompt = SystemMessagePromptTemplate.from_template(context_template)

human_template="Please generate 15 questions . The questions should what the topic is about and allow you to flesh out each of these questions into a 500 word answer that covers all elements of the question. \n\n Topic: \n\n{topic_description} \
\n\n  Questions:"

human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=human_template,
            input_variables=["topic_description"],
        )
    )

chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt,
                                                         human_message_prompt])

chat_llm = ChatOpenAI(model_name="gpt-3.5-turbo",
                  temperature=0.2)

question_chain = LLMChain(llm=chat_llm, prompt=chat_prompt_template)

In [22]:
TOPIC_DESCRIPTION = "The pros and cons of regulating AI research and development"

In [23]:
%%time
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = question_chain.run(TOPIC_DESCRIPTION)

    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
    print(f"Calc Total Cost (USD): ${(cb.prompt_tokens/1000)*0.003 + (cb.completion_tokens/1000)*0.004}")

Total Tokens: 360
Prompt Tokens: 108
Completion Tokens: 252
Total Cost (USD): $0.000666
Calc Total Cost (USD): $0.001332
CPU times: user 41.2 ms, sys: 7.16 ms, total: 48.3 ms
Wall time: 4.37 s


In [24]:
print(wrap_text(output))

# 1. What are the potential benefits of regulating AI research and development? # 2. How can regulations help ensure the
ethical use of AI technology? # 3. What are the risks associated with unregulated AI research and development? # 4. How
can regulations impact the pace of AI innovation? # 5. What are the challenges of implementing effective regulations in
the field of AI? # 6. How do different countries approach the regulation of AI research and development? # 7. What role
do ethics play in the regulation of AI technology? # 8. How can regulations address concerns about AI bias and
discrimination? # 9. What are the implications of regulating AI on job displacement and the economy? # 10. How do
privacy concerns intersect with the regulation of AI technology? # 11. What are the arguments against strict regulations
on AI research and development? # 12. How can regulations foster international cooperation in the field of AI? # 13.
What are the key considerations for policymakers when de

In [25]:
write_string_to_file(f'Questions-{TOPIC_DESCRIPTION}.txt', output)

In [26]:
num_tokens_from_string(output, "gpt-3.5-turbo")

252

In [27]:
questions = output

## Generate the article

In [28]:
context_template="You are a master writer that specializes in creating long 5,000 word articles. Please use all your expertise to approach this task. Output your content in markdown format and include titles and subtitles where relevant."

system_message_prompt = SystemMessagePromptTemplate.from_template(context_template)

human_template="Please take this list of questions and step by step go through each of them writing 4 paragraphs for each question. Include a subtitle for the question that rephrases the question. \n\n QUESTIONS: \n\n{questions} \
\n\n  ARTICLE:"

human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=human_template,
            input_variables=["questions"],
        )
    )

chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt,
                                                         human_message_prompt])

chat_llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k",
                  temperature=0.2)

question_chain = LLMChain(llm=chat_llm, prompt=chat_prompt_template)

In [29]:
%%time
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = question_chain.run(TOPIC_DESCRIPTION)

    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
    print(f"Calc Total Cost (USD): ${(cb.prompt_tokens/1000)*0.003 + (cb.completion_tokens/1000)*0.004}")

Total Tokens: 1352
Prompt Tokens: 107
Completion Tokens: 1245
Total Cost (USD): $0.005301000000000001
Calc Total Cost (USD): $0.005301000000000001
CPU times: user 91.5 ms, sys: 10.2 ms, total: 102 ms
Wall time: 13.2 s


In [30]:
print(wrap_text(output))

## The Pros and Cons of Regulating AI Research and Development  ### Introduction  Artificial Intelligence (AI) has
become one of the most transformative technologies of our time, with the potential to revolutionize various industries
and improve our daily lives. However, as AI continues to advance at an unprecedented pace, concerns have been raised
about the ethical implications and potential risks associated with its development. This has led to a debate on whether
there should be regulations imposed on AI research and development. In this article, we will explore the pros and cons
of regulating AI research and development, considering both the benefits and drawbacks of such regulations.  ### Pros of
Regulating AI Research and Development  #### Ensuring Ethical Development  One of the main arguments in favor of
regulating AI research and development is the need to ensure ethical practices. AI systems have the potential to make
decisions and take actions that can have significant conse

In [53]:
write_string_to_file(f'Questions-{TOPIC_DESCRIPTION}.txt', output)

In [54]:
num_tokens_from_string(output, "gpt-3.5-turbo")

998

In [55]:
questions = output

## Generate the article

In [56]:
context_template="You are a master writer that specializes in creating long 5,000 word articles. Please use all your expertise to approach this task. Output your content in markdown format and include titles and subtitles where relevant."

system_message_prompt = SystemMessagePromptTemplate.from_template(context_template)

human_template="Please take this list of questions and step by step go through each of them writing 4 paragraphs for each question. Include a subtitle for the question that rephrases the question. \n\n QUESTIONS: \n\n{questions} \
\n\n  ARTICLE:"

human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=human_template,
            input_variables=["questions"],
        )
    )

chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt,
                                                         human_message_prompt])

chat_llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k",
                  temperature=0.2)

article_chain = LLMChain(llm=chat_llm, prompt=chat_prompt_template)

In [35]:
%%time
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = article_chain.run(TOPIC_DESCRIPTION)

    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
    print(f"Calc Total Cost (USD): ${(cb.prompt_tokens/1000)*0.003 + (cb.completion_tokens/1000)*0.004}")

Total Tokens: 1293
Prompt Tokens: 107
Completion Tokens: 1186
Total Cost (USD): $0.005065
Calc Total Cost (USD): $0.005065
CPU times: user 109 ms, sys: 8.84 ms, total: 118 ms
Wall time: 14.4 s


In [57]:
print(wrap_text(output))

# The Pros and Cons of Regulating AI Research and Development  ## Introduction  Artificial Intelligence (AI) has become
an integral part of our lives, impacting various sectors such as healthcare, finance, and transportation. As AI
continues to advance rapidly, there is an ongoing debate about the need for regulation in AI research and development.
This article explores the pros and cons of regulating AI, addressing key questions and concerns surrounding this topic.
## Pros of Regulating AI Research and Development  ### 1. Ensuring Ethical Development  Regulating AI research and
development can help ensure ethical practices are followed. By establishing guidelines and standards, we can prevent the
creation of AI systems that may cause harm or infringe upon human rights. Ethical considerations such as fairness,
transparency, and accountability can be integrated into the development process, promoting responsible AI deployment.
### 2. Mitigating Bias and Discrimination  AI systems are on

In [58]:
write_string_to_file(f'Article-{TOPIC_DESCRIPTION}.txt', output)

In [59]:
num_tokens_from_string(output, "gpt-3.5-turbo")

998